In [1]:
!pip3 install -U sentence-transformers

    100% |████████████████████████████████| 61kB 988kB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/a3/78/92cedda05552398352ed9784908b834ee32a0bd071a9b32de287327370b7/transformers-2.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4a/1c/6359be64e8301b84160f6f6f7936bbfaaa5e9a4eab6cbc681db07600b949/tqdm-4.45.0-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 752.0MB 2.5kB/s ta 0:00:0122
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5e/d8/312e03adf4c78663e17d802fe2440072376fee46cada1404f1727ed77a32/scikit_learn-0.22.2.post1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m

In [3]:
import os
import numpy as np
import pandas as pd
#from google.colab import drive
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from sentence_transformers import SentenceTransformer, LoggingHandler
import logging

[nltk_data] Downloading package punkt to /home/adnaks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
MAX_LEN=512

In [5]:
def split_document_to_chunks(row, field_name, chunk_len):
    text = row[field_name]
    if(text!='[]'):
        label = row['label']
        sentences = sent_tokenize(text)
        output = []
        for i in range(0,len(sentences), chunk_len):
            if(i+chunk_len < len(sentences)):
                chunk = ''.join(sentences[i:i+chunk_len])
            else:
                chunk = ''.join(sentences[i:len(sentences)])
            output.append((chunk,label))
        return output
def prepare_data_set(data_set):
    chunked_text_labels = data_set.apply(split_document_to_chunks, args=('filteredtext_aapl', 3), axis=1)
    X=[]
    y=[]
    for chunks in chunked_text_labels:
        if(chunks is not None):
            for chunk in chunks:
                X.append(chunk[0])
                y.append(chunk[1])
    return X,y

In [6]:
df = pd.read_csv('../Data/final_dataset_joined_aapl240_onlyMentions.csv')

In [7]:
del df['Unnamed: 0']
train_df = df[df.stock_time <= "2018-12-01 00:00:00"]
test_df = df[df.stock_time > "2018-12-01 00:00:00"]
#df = df[df.filteredtext_aapl != '[]']
#df.index = range(df.shape[0])
#df['encoded_aapl'] = df.filteredtext_aapl.apply(text_encoder, args=(tokenizer, MAX_LEN))
#train_df = df[df.stock_time <= "2018-12-01 00:00:00"]
#test_df = df[df.stock_time > "2018-12-01 00:00:00"]

In [8]:
'''
train_input = aapl_news[train_df.index]
train_label = labels[train_df.index]
test_input = aapl_news[test_df.index]
test_label = labels[test_df.index]
'''
train_input, train_label = prepare_data_set(train_df)
test_input, test_label = prepare_data_set(test_df)
#train_label[train_label == -1] = 0
#test_label[test_label == -1] = 0

In [9]:
train_label = np.array(train_label)
test_label = np.array(test_label)
train_label[train_label == -1] = 0
test_label[test_label == -1] = 0
len(train_input), len(train_label)

(59278, 59278)

In [10]:
len(test_input), len(test_label)

(13349, 13349)

In [15]:
model = SentenceTransformer('bert-base-nli-mean-tokens')


  0%|          | 0.00/405M [00:00<?, ?B/s]
  0%|          | 16.4k/405M [00:00<1:07:00, 101kB/s]
  0%|          | 32.8k/405M [00:00<1:07:47, 99.6kB/s]
  0%|          | 65.5k/405M [00:00<58:18, 116kB/s]   
  0%|          | 81.9k/405M [00:00<1:06:06, 102kB/s]
  0%|          | 115k/405M [00:00<58:54, 115kB/s]   
  0%|          | 147k/405M [00:01<53:53, 125kB/s]
  0%|          | 180k/405M [00:01<47:54, 141kB/s]
  0%|          | 213k/405M [00:01<48:40, 139kB/s]
  0%|          | 262k/405M [00:01<42:33, 159kB/s]
  0%|          | 311k/405M [00:01<38:09, 177kB/s]
  0%|          | 360k/405M [00:02<35:09, 192kB/s]
  0%|          | 410k/405M [00:02<31:25, 215kB/s]
  0%|          | 459k/405M [00:02<32:02, 211kB/s]
  0%|          | 524k/405M [00:02<27:34, 245kB/s]
  0%|          | 590k/405M [00:02<26:47, 252kB/s]
  0%|          | 672k/405M [00:03<23:48, 283kB/s]
  0%|          | 754k/405M [00:03<20:54, 322kB/s]
  0%|          | 836k/405M [00:03<20:30, 329kB/s]
  0%|          | 918k/405M [00:03<19:23

 14%|█▍        | 56.3M/405M [00:20<01:12, 4.78MB/s]
 14%|█▍        | 56.8M/405M [00:20<01:12, 4.81MB/s]
 14%|█▍        | 57.3M/405M [00:20<01:12, 4.79MB/s]
 14%|█▍        | 57.8M/405M [00:20<01:11, 4.83MB/s]
 14%|█▍        | 58.3M/405M [00:20<01:11, 4.85MB/s]
 15%|█▍        | 58.8M/405M [00:20<01:11, 4.82MB/s]
 15%|█▍        | 59.3M/405M [00:20<01:12, 4.79MB/s]
 15%|█▍        | 59.8M/405M [00:20<01:10, 4.88MB/s]
 15%|█▍        | 60.3M/405M [00:21<01:10, 4.92MB/s]
 15%|█▌        | 60.8M/405M [00:21<01:11, 4.79MB/s]
 15%|█▌        | 61.3M/405M [00:21<01:09, 4.93MB/s]
 15%|█▌        | 61.8M/405M [00:21<01:10, 4.86MB/s]
 15%|█▌        | 62.3M/405M [00:21<01:13, 4.64MB/s]
 16%|█▌        | 62.9M/405M [00:21<01:08, 4.98MB/s]
 16%|█▌        | 63.4M/405M [00:21<01:11, 4.77MB/s]
 16%|█▌        | 63.9M/405M [00:21<01:11, 4.76MB/s]
 16%|█▌        | 64.4M/405M [00:21<01:11, 4.78MB/s]
 16%|█▌        | 64.9M/405M [00:21<01:09, 4.88MB/s]
 16%|█▌        | 65.4M/405M [00:22<01:08, 4.93MB/s]
 16%|█▋     

 34%|███▍      | 138M/405M [00:36<00:57, 4.65MB/s]
 34%|███▍      | 139M/405M [00:36<00:59, 4.46MB/s]
 34%|███▍      | 139M/405M [00:36<00:55, 4.77MB/s]
 34%|███▍      | 140M/405M [00:36<00:57, 4.65MB/s]
 35%|███▍      | 140M/405M [00:36<00:58, 4.51MB/s]
 35%|███▍      | 141M/405M [00:36<00:55, 4.81MB/s]
 35%|███▍      | 141M/405M [00:37<00:57, 4.61MB/s]
 35%|███▍      | 142M/405M [00:37<00:57, 4.62MB/s]
 35%|███▌      | 142M/405M [00:37<00:56, 4.69MB/s]
 35%|███▌      | 143M/405M [00:37<00:56, 4.67MB/s]
 35%|███▌      | 143M/405M [00:37<00:53, 4.86MB/s]
 35%|███▌      | 144M/405M [00:37<00:55, 4.71MB/s]
 36%|███▌      | 144M/405M [00:37<00:55, 4.73MB/s]
 36%|███▌      | 145M/405M [00:37<00:53, 4.84MB/s]
 36%|███▌      | 145M/405M [00:37<00:54, 4.76MB/s]
 36%|███▌      | 146M/405M [00:37<00:54, 4.80MB/s]
 36%|███▌      | 146M/405M [00:38<00:53, 4.87MB/s]
 36%|███▌      | 147M/405M [00:38<00:53, 4.86MB/s]
 36%|███▋      | 147M/405M [00:38<00:52, 4.92MB/s]
 36%|███▋      | 148M/405M [00:

 55%|█████▍    | 221M/405M [00:52<00:41, 4.41MB/s]
 55%|█████▍    | 222M/405M [00:52<00:39, 4.59MB/s]
 55%|█████▍    | 222M/405M [00:52<00:41, 4.45MB/s]
 55%|█████▍    | 223M/405M [00:52<00:39, 4.59MB/s]
 55%|█████▌    | 223M/405M [00:52<00:39, 4.60MB/s]
 55%|█████▌    | 224M/405M [00:53<00:41, 4.40MB/s]
 55%|█████▌    | 224M/405M [00:53<00:38, 4.65MB/s]
 55%|█████▌    | 225M/405M [00:53<00:39, 4.55MB/s]
 56%|█████▌    | 225M/405M [00:53<00:39, 4.61MB/s]
 56%|█████▌    | 226M/405M [00:53<00:38, 4.69MB/s]
 56%|█████▌    | 226M/405M [00:53<00:38, 4.64MB/s]
 56%|█████▌    | 227M/405M [00:53<00:37, 4.73MB/s]
 56%|█████▌    | 227M/405M [00:53<00:37, 4.73MB/s]
 56%|█████▌    | 227M/405M [00:53<00:38, 4.66MB/s]
 56%|█████▌    | 228M/405M [00:54<00:38, 4.61MB/s]
 56%|█████▋    | 228M/405M [00:54<00:37, 4.77MB/s]
 56%|█████▋    | 229M/405M [00:54<00:37, 4.75MB/s]
 57%|█████▋    | 229M/405M [00:54<00:37, 4.70MB/s]
 57%|█████▋    | 230M/405M [00:54<00:36, 4.82MB/s]
 57%|█████▋    | 230M/405M [00:

 75%|███████▌  | 304M/405M [01:08<00:23, 4.34MB/s]
 75%|███████▌  | 305M/405M [01:08<00:23, 4.19MB/s]
 75%|███████▌  | 305M/405M [01:08<00:24, 4.17MB/s]
 75%|███████▌  | 306M/405M [01:08<00:22, 4.34MB/s]
 76%|███████▌  | 306M/405M [01:09<00:30, 3.29MB/s]
 76%|███████▌  | 307M/405M [01:09<00:29, 3.32MB/s]
 76%|███████▌  | 307M/405M [01:09<00:25, 3.83MB/s]
 76%|███████▌  | 308M/405M [01:09<00:26, 3.62MB/s]
 76%|███████▌  | 308M/405M [01:09<00:25, 3.77MB/s]
 76%|███████▌  | 309M/405M [01:09<00:23, 4.08MB/s]
 76%|███████▌  | 309M/405M [01:09<00:23, 4.08MB/s]
 76%|███████▋  | 309M/405M [01:09<00:23, 4.06MB/s]
 76%|███████▋  | 310M/405M [01:10<00:22, 4.16MB/s]
 77%|███████▋  | 310M/405M [01:10<00:22, 4.23MB/s]
 77%|███████▋  | 311M/405M [01:10<00:22, 4.23MB/s]
 77%|███████▋  | 311M/405M [01:10<00:21, 4.31MB/s]
 77%|███████▋  | 312M/405M [01:10<00:21, 4.32MB/s]
 77%|███████▋  | 312M/405M [01:10<00:21, 4.38MB/s]
 77%|███████▋  | 313M/405M [01:10<00:20, 4.43MB/s]
 77%|███████▋  | 313M/405M [01:

 95%|█████████▍| 383M/405M [01:25<00:04, 4.85MB/s]
 95%|█████████▍| 384M/405M [01:25<00:04, 4.61MB/s]
 95%|█████████▍| 384M/405M [01:25<00:04, 4.76MB/s]
 95%|█████████▍| 385M/405M [01:25<00:04, 4.88MB/s]
 95%|█████████▌| 385M/405M [01:25<00:04, 4.71MB/s]
 95%|█████████▌| 386M/405M [01:25<00:04, 4.84MB/s]
 95%|█████████▌| 386M/405M [01:25<00:03, 4.88MB/s]
 95%|█████████▌| 387M/405M [01:26<00:03, 4.76MB/s]
 96%|█████████▌| 387M/405M [01:26<00:03, 5.03MB/s]
 96%|█████████▌| 388M/405M [01:26<00:03, 4.86MB/s]
 96%|█████████▌| 388M/405M [01:26<00:03, 4.81MB/s]
 96%|█████████▌| 389M/405M [01:26<00:03, 5.00MB/s]
 96%|█████████▌| 389M/405M [01:26<00:03, 4.88MB/s]
 96%|█████████▌| 390M/405M [01:26<00:03, 4.79MB/s]
 96%|█████████▋| 390M/405M [01:26<00:03, 4.78MB/s]
 96%|█████████▋| 391M/405M [01:26<00:02, 4.93MB/s]
 97%|█████████▋| 391M/405M [01:26<00:02, 4.85MB/s]
 97%|█████████▋| 392M/405M [01:27<00:02, 4.78MB/s]
 97%|█████████▋| 393M/405M [01:27<00:02, 4.91MB/s]
 97%|█████████▋| 393M/405M [01:

In [16]:
%%time
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

CPU times: user 367 ms, sys: 7.37 ms, total: 374 ms
Wall time: 72 ms


In [18]:
sentence_embeddings[2].shape

(768,)

In [20]:
len(train_input)

59278

In [22]:
%%time
bert_embeddings = model.encode(train_input)

CPU times: user 6h 50min 37s, sys: 6min 53s, total: 6h 57min 31s
Wall time: 1h 17min 58s


In [23]:
len(bert_embeddings)

59278

In [25]:
X_train = np.array(bert_embeddings)

In [26]:
X_train.shape

(59278, 768)

In [27]:
np.save('../Data/bert_embeddings_train.npy', X_train)

In [29]:
%%time
X_test = np.array(model.encode(test_input))

CPU times: user 1h 48min 4s, sys: 2min 40s, total: 1h 50min 44s
Wall time: 18min 54s


In [30]:
np.save('../Data/bert_embeddings_test.npy', X_test)

In [36]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, train_label)

CPU times: user 9min 9s, sys: 1min, total: 10min 9s
Wall time: 53 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
#y_pred = clf.predict(X_test)
clf.score(X_test, test_label)

0.5326241666042401

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, train_label)
rf.score(X_test, test_label)

0.5333732863884936

In [41]:
%%time
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, train_label)
svc.score(X_test, test_label)

CPU times: user 1h 8min 27s, sys: 618 ms, total: 1h 8min 28s
Wall time: 1h 8min 28s


0.5320997827552626

In [43]:
ypred = svc.predict(X_test)

In [ ]:
from sklearn.metrics import precision_Score, recall_score, confusion matrix
print(precision_score(test_label, ypred))
print(recall_score(test_label, ypred))